# Modelo Oculto Markov - Post Tagging

En el contexto del post-tagging, los estados del HMM representan las etiquetas gramaticales de las palabras, las transiciones representan la dependencia entre las etiquetas gramaticales de las palabras consecutivas y las emisiones representan la probabilidad de que una palabra dada se observe en un estado determinado.

Para realizar el post-tagging con un HMM, se sigue el siguiente procedimiento:

- *Entrenamiento del modelo*: se entrena con un conjunto de datos de oraciones etiquetadas, aprendiendo las probabilidades de transición y emisión

- *Predicción de etiquetas*: para una nueva oración sin etiquetar, el modelo predice la secuencia de etiquetas gramaticales más probable para la oración, utilizando el algoritmo de Viterbi

Importación de librerías y carga del corpus `brown`

In [1]:
import warnings

import nltk
import numpy as np
from hmmlearn import hmm

warnings.filterwarnings("ignore")

from nltk.corpus import brown # corpus con etiquetado

# Cargar las sentencias etiquetadas del corpus brown
tagged_sentences = brown.tagged_sents(tagset='english')


Se crean dos diccionarios para mapear cada palabra con su target (post-tag): 
- *word2idx* para mapear palabras a índices numéricos 
- *tag2idx* para mapear etiquetas gramaticales a índices

In [ ]:
# Crear un diccionario de palabras y un diccionario de etiquetas
word2idx = {}
tag2idx = {}

# Iterar sobre las sentencias etiquetadas para construir los diccionarios
for sentence in tagged_sentences:
    for word, tag in sentence:
        if word.lower() not in word2idx:
            word2idx[word.lower()] = len(word2idx)
        if tag not in tag2idx:
            tag2idx[tag] = len(tag2idx)

# Estos diccionarios serán útiles para convertir palabras y tags en índices numéricos que nuestro modelo HMM pueda entender.

Se crean los datos de entrenamiento. 

En este caso, se itera sobre las sentencias etiquetadas del corpus para llenar *words_train* (lista de palabras en minúsculas) y *tags_train* (lista de etiquetas)

In [ ]:
# Conjunto de entrenamiento
words_train = [] # Lista de palabras (en minúscualas por lower)  
tags_train = [] # Lista de etiquetas
for sentence in tagged_sentences:
    words, tags = zip(*sentence)
    words_train.append([word.lower() for word in words])
    tags_train.append(tags)

Creación y entrenamiento del modelo: se crea y entrena un modelo HMM (MultinomialHMM) usando los datos de entrenamiento convertidos a índices numéricos

In [2]:
# Creación y entrenamiento del modelo HMM
model = hmm.MultinomialHMM(n_components=len(tag2idx), init_params="ste") # estados ocultos como número de etiquetas
model.fit(
    X=np.array([word2idx[word] for words in words_train for word in words]).reshape(-1, 1),
    lengths=[len(words) for words in words_train]
) # El entrenamiento se hace converiendo a índices las palabras

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


MultinomialHMM(n_trials=array([    0,     1,     2, ...,    44, 49814,    24]),
               random_state=RandomState(MT19937) at 0x1E9D62E8540)

Se obtiene la función de etiquetado **post_tag**. 
En esta función se etiqueta una nueva sentencia. Convierte las palabras de la sentencia en índices, predice las etiquetas usando el modelo HMM, y devuelve las palabras con sus etiquetas correspondientes.

In [3]:
# Función para realizar post-tagging en una nueva sentencia en inglés
def post_tag(model, sentence, word2idx, tag2idx):
    
    # Convertir las palabras de la sentencia a índices
    word_idxs = [word2idx[word.lower()] for word in sentence if word.lower() in word2idx]
    
    # Si no hay palabras conocidas, devolver None
    if len(word_idxs) == 0:
        return None
    
    # Realizar post-tagging utilizando el modelo HMM
    predicted_tags = model.predict(np.array(word_idxs).reshape(-1, 1))
    
    # Convertir los índices de etiquetas a etiquetas POS
    predicted_tags = [list(tag2idx.keys())[list(tag2idx.values()).index(tag)] for tag in predicted_tags]
    
    return list(zip(sentence, predicted_tags))

Finalmente, realizamos una predicción. Usamos como ejemplo la frase: `I love Python`

In [4]:
sentence = "I love Python"
predicted_tags = post_tag(model, sentence.split(), word2idx, tag2idx)
print(f"Post-tagging de la oración: {predicted_tags}")

Post-tagging de la oración: [('I', 'UNK'), ('love', 'UNK'), ('Python', 'UNK')]
